In [ ]:
import pandas as pd
from util_func import *
from util_func import (
    out_dir,
    survey_processed_dir,
    tour_extract_wkday_dir,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "05_TNCTrips_Misc.xlsx"

In [ ]:
## Process trip records
raw_trips = pd.read_csv(survey_processed_dir / "trip.csv")
raw_trips = raw_trips[
    [
        "hh_id",
        "person_num",
        "trip_num",
        "mode_uber",
        "mode_lyft",
        "tnc_decide",
        "tnc_schedule",
        "tnc_replace",
        "tnc_pooled",
        "mode_type_imputed",
    ]
]
raw_trips = raw_trips.rename(
    columns={"hh_id": "hhno", "person_num": "pno", "trip_num": "tsvid"}
)

trip_df = pd.read_csv(tour_extract_wkday_dir / "trips-assign_day.csv")
trip_df = link_dt(trip_df)
trip_df = trip_df.loc[trip_df["mode"] == 9,]
trip_df = trip_dist_map(trip_df)
trip_df = trip_df.merge(raw_trips, how="left")
trip_df = trip_df[trip_df["mode_type_imputed"] != 4]  # remove taxi trips

trip_df["tnc_type"] = 3  # prem/other
trip_df.loc[(trip_df["mode_uber"] == 1) | (trip_df["mode_lyft"] == 1), "tnc_type"] = (
    1  # pooled
)
trip_df.loc[(trip_df["mode_uber"] == 2) | (trip_df["mode_lyft"] == 2), "tnc_type"] = (
    2  # regular
)

trip_df["count"] = 1
trip_df = trip_df[(trip_df["trexpfac"] > 0) & (trip_df["mode"] > 0)]
trip_df = trip_df[(trip_df["otaz"] > 0) & (trip_df["dtaz"] > 0)]

In [ ]:
col_dict = {
    "altmode": {
        "desc": "TNC_Alt",
        "col": "tnc_replace",
        "vals": list(range(1, 8)) + [9, 10, 997, 995],
        "labels": [
            "1_Taxi",
            "2_DA",
            "3_SR",
            "4_Trn",
            "5_WkBk",
            "6_ODes",
            "7_NoTrp",
            "90_BART",
            "91_OTrn",
            "92_Oth",
            "93_Miss",
        ],
    },
    "when": {
        "desc": "TNC_When",
        "col": "tnc_decide",
        "vals": list(range(1, 6)) + [995],
        "labels": [
            "1_RightBef",
            "2_Hour",
            "3_SameDay",
            "4_PrevDay",
            "5_2+Days",
            "6_Miss",
        ],
    },
    "sched": {
        "desc": "TNC_Sched",
        "col": "tnc_schedule",
        "vals": list(range(2)) + [995],
        "labels": ["1_No", "2_Yes", "3_Miss"],
    },
    "tncmode": {
        "desc": "TNC_Mode",
        "col": "tnc_type",
        "vals": range(1, 4),
        "labels": ["2_Pool", "1_Reg", "3_PremOth"],
    },
}

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
for key in ["when", "sched", "altmode", "tncmode"]:
    d1_dict = col_dict[key]
    row = 0
    sname = d1_dict["desc"]

    title = "TNC Trips by " + d1_dict["desc"]
    tab = prep_data_1d(
        trip_df,
        d1_dict["desc"],
        d1_dict["col"],
        "trexpfac",
        d1_dict["vals"],
        d1_dict["labels"],
    )
    row = write_to_excel(tab, sname, title, row)

    title = "Column Shares by " + d1_dict["desc"]
    row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)

    tab2 = tab.copy()
    tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

    title = "Column Shares 95% CI by " + d1_dict["desc"]
    row = write_to_excel(getSharesIdxCI95(tab.copy()), sname, title, row)

    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()